<a href="https://colab.research.google.com/github/Elwing-Chou/ml0223/blob/main/titanic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from urllib.request import urlretrieve
url = "https://github.com/Elwing-Chou/ml0223/raw/main/train.csv"
urlretrieve(url, "train.csv")
url = "https://github.com/Elwing-Chou/ml0223/raw/main/test.csv"
urlretrieve(url, "test.csv")

In [ ]:
import pandas as pd
train_df = pd.read_csv("train.csv", encoding="utf-8")
test_df = pd.read_csv("test.csv", encoding="utf-8")

In [ ]:
test_df

In [ ]:
datas = pd.concat([train_df, test_df], axis=0, ignore_index=True)
datas

In [ ]:
s = datas.isna().sum()
# Series[True/False]
s[s > 0].sort_values(ascending=False)

In [ ]:
datas = datas.drop(["PassengerId", "Survived"], axis=1)
datas

In [ ]:
# 數值類型: 中位數
med = datas.median().drop("Pclass")
datas = datas.fillna(med)

In [ ]:
# 類別類型: 最常出現
most = datas["Embarked"].value_counts().idxmax()
datas["Embarked"] = datas["Embarked"].fillna(most)

In [ ]:
s = datas.isna().sum()
# Series[True/False]
s[s > 0].sort_values(ascending=False)

In [ ]:
def cabin(s):
    if pd.isna(s):
        return None
    else:
        return s[0]
datas["Cabin"] = datas["Cabin"].apply(cabin)

In [ ]:
def name(s):
    mid = s.split(",")[-1].split(".")[0]
    mid = mid.strip()
    return mid
count = datas["Name"].apply(name).value_counts()
reserved = count[:4].index
def name2(s):
    mid = s.split(",")[-1].split(".")[0]
    mid = mid.strip()
    if mid in reserved:
        return mid
    else:
        return None
datas["Name"] = datas["Name"].apply(name2)

In [ ]:
datas

In [ ]:
dic = datas["Ticket"].value_counts()
def ticket(t):
    return dic[t]
datas["Ticket"] = datas["Ticket"].apply(ticket)

In [ ]:
datas = pd.get_dummies(datas)
datas = pd.get_dummies(datas, columns=["Pclass"])
datas

In [ ]:
datas["Family"] = datas["SibSp"] + datas["Parch"]
datas

In [ ]:
y = train_df["Survived"]
# .iloc: [1st row, 2nd rows....]
x = datas.iloc[:len(train_df)]
predict = datas.iloc[len(train_df):]

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
clf = RandomForestClassifier()
params = {
    "n_estimators":range(50, 53, 2),
    "max_depth":range(5, 7)
}
grid = GridSearchCV(clf, params, cv=10, n_jobs=-1, scoring="neg_log_loss")
grid.fit(x, y)
print(grid.best_score_)
print(grid.best_params_)

In [ ]:
import numpy as np
from sklearn.model_selection import cross_val_score
clf = RandomForestClassifier(n_estimators=53, max_depth=7)
scores = cross_val_score(clf, x, y, cv=10, n_jobs=-1)
print(scores)
print(np.average(scores))
clf = RandomForestClassifier(n_estimators=53, max_depth=7)
scores = cross_val_score(clf, x, y, cv=10, n_jobs=-1, scoring="neg_log_loss")
print(scores)
print(np.average(scores))

In [ ]:
clf = RandomForestClassifier(n_estimators=53, max_depth=7)
clf.fit(x, y)
pre = clf.predict(predict)
ans = pd.DataFrame({
    "PassengerId":test_df["PassengerId"],
    "Survived":pre
})
ans.to_csv("rf.csv", encoding="utf-8", index=False)

In [ ]:
import pandas as pd
pd.DataFrame(clf.feature_importances_, 
       columns=["Importance"],
       index=datas.columns).sort_values(by="Importance", ascending=False)

In [ ]:
dt = clf.estimators_[0]

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
datas_scale = scaler.fit_transform(datas)
datas_scale = pd.DataFrame(datas_scale, columns=datas.columns)
datas_scale

In [ ]:
x_scale = datas_scale.iloc[:len(train_df)]
predict_scale = datas_scale.iloc[len(train_df):]

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
clf = KNeighborsClassifier()
params = {
    "n_neighbors":range(5, 200),
}
grid = GridSearchCV(clf, params, cv=10, n_jobs=-1, scoring="neg_log_loss")
grid.fit(x_scale, y)
print(grid.best_score_)
print(grid.best_params_)

In [ ]:
clf = KNeighborsClassifier(n_neighbors=11)
clf.fit(x_scale, y)
pre = clf.predict(predict_scale)
ans = pd.DataFrame({
    "PassengerId":test_df["PassengerId"],
    "Survived":pre
})
ans.to_csv("knn.csv", encoding="utf-8", index=False)